In [83]:
import pandas as pd
import numpy as np
import pandas_profiling
#import seaborn as sns
import wget
import tabula
from datetime import date
import datetime
from pathlib import Path

import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
#pio.templates.default = "plotly_dark"
from plotly.subplots import make_subplots

pd.set_option('display.max_columns',99)
pd.set_option('display.max_rows',200)

today = date.today()

In [7]:
#submission = pd.read_csv('submission.csv')
#train = pd.read_csv('train.csv')

In [46]:
datedf = '3/30/20'
datedf = datetime.datetime.strptime(datedf, '%m/%d/%y')
print(datedf)

2020-03-30 00:00:00


### **I. NYC Specific Data**

In [47]:
folderpath = Path('/Users/Emanuel/Documents/2019 Plan/NYC Data Science Academy/Post Bootcamp/COVID-19/Forecasting/covid19-global-forecasting-week-1/nyc')
# file_to_open = folderpath / 'covid-19-daily-data-summary-hospitalizations.pdf'
# files = [folderpath / 'covid-19-daily-data-summary-hospitalizations.pdf',folderpath / 'covid-19-daily-data-summary-deaths.pdf',folderpath / 'covid-19-daily-data-summary.pdf']

# for i in range(len(files)):
    
#     tabula.convert_into(files[i],names[i], output_format="csv", pages='all')

In [86]:
# convert PDFs into CSV, read CSVs into DataFrames.

urls = ['https://www1.nyc.gov/assets/doh/downloads/pdf/imm/covid-19-daily-data-summary-hospitalizations.pdf',
        "https://www1.nyc.gov/assets/doh/downloads/pdf/imm/covid-19-daily-data-summary-deaths.pdf",
        'https://www1.nyc.gov/assets/doh/downloads/pdf/imm/covid-19-daily-data-summary.pdf']

names = ['/Users/Emanuel/Documents/2019 Plan/NYC Data Science Academy/Post Bootcamp/COVID-19/Forecasting/covid19-global-forecasting-week-1/nyc/NYCH.csv',
         '/Users/Emanuel/Documents/2019 Plan/NYC Data Science Academy/Post Bootcamp/COVID-19/Forecasting/covid19-global-forecasting-week-1/nyc/NYCD.csv',
         '/Users/Emanuel/Documents/2019 Plan/NYC Data Science Academy/Post Bootcamp/COVID-19/Forecasting/covid19-global-forecasting-week-1/nyc/NYCC.csv']

for i in range(len(urls)):
    
    tabula.convert_into(urls[i],names[i], output_format="csv", pages='all')
    
nycd = pd.read_csv(folderpath / 'NYCD.csv', header = None)
nych = pd.read_csv(folderpath / 'NYCH.csv', header = None)
nycc = pd.read_csv(folderpath / 'NYCC.csv')

In [85]:
nycc

,.,Unnamed: 1,Total Cases
0,Total,NaN,40900
1,Median Age (Range),NaN,48 (0-105)
2,Age Group,NaN,NaN
3,- 0 to 17,NaN,748 (2%)
4,- 18 to 44,NaN,17038 (42%)
5,- 45 to 64,NaN,14395 (35%)
6,- 65 to 74,NaN,4883 (12%)
7,- 75 and over,NaN,3739 (9%)
8,- Unknown,NaN,97
9,Age 50 and over,NaN,NaN


In [35]:
nych

,0,1,2
0,Age Group,NaN,NaN
1,- 0 to 17,72 (10%),714.0
2,- 18 to 44,1448 (9%),16028.0
3,- 45 to 64,2887 (22%),13344.0
4,- 65 to 74,1612 (36%),4496.0
5,- 75 and over,1722 (50%),3410.0
6,- Unknown,0 (0%),95.0
7,Sex,NaN,NaN
8,- Female,3130 (18%),16920.0
9,- Male,4610 (22%),21120.0


In [36]:
nycd

,0,1,2,3,4
0,Age Group,NaN,NaN,NaN,NaN
1,- 0 to 17,1.0,0.0,0.0,1.0
2,- 18 to 44,39.0,2.0,13.0,54.0
3,- 45 to 64,170.0,9.0,37.0,216.0
4,- 65 to 74,167.0,0.0,48.0,215.0
5,- 75 and over,327.0,3.0,98.0,428.0
6,Sex,NaN,NaN,NaN,NaN
7,- Female,270.0,4.0,70.0,344.0
8,- Male,433.0,10.0,126.0,569.0
9,- Unknown,1.0,0.0,0.0,1.0


**Helper functions when reading in the PDFs from NYC.gov**

In [56]:
def getIndices(df, value):
    '''Given a dataframe and a value, returns the index of the value
       within the dataframe, otherwise returns -1.
    '''
    #get bool dataframe showing whether the object is actually in the dataframe or not.
    booldf = df.isin([value])
    #show which columns have the object with bool True
    boolseries = booldf.any()
    #shrink the result so that only True shows up. Store the successful columns in a list.
    colnames = list(boolseries[boolseries == True].index)
    #iterate through the columns to check
    
    results = []
    
    for col in colnames:
        #store all matching rows into a list
        rows = list(booldf[col][booldf[col] == True].index)
        for row in rows:
            results.append((row,col)) #append tuple of row, column to result list.
    
    return results
    

#format the value column.
def countform(rowitem):
    try:
        return int(rowitem[:rowitem.find('(')-1])
    except:
        return int(rowitem)
    

In [57]:
dcats = ['Age Group','Sex','Borough','Total']
hcats = ['Age Group','Sex','Borough','Total']
ccats = ['Age Group','Age 50 and over','Sex','Borough','Deaths']

In [58]:
#slice the original dataframe and get new dataframes.

def getdfs(df,categories):
    
    for i in range(len(categories)-1):
    
        if df.equals(nych):    
            temp = df[getIndices(df,categories[i])[0][0]:getIndices(df,categories[i+1])[0][0]].rename({0:categories[i], 1:'Ever Hospitalized Cases', 2:'Total Cases'}, axis = 1).reset_index(drop=True)
        elif df.equals(nycd):
            temp = df[getIndices(df,categories[i])[0][0]:getIndices(df,categories[i+1])[0][0]].rename({0:categories[i],1:'Underlying Conditions',
                                                                                                       2:'No Underlying Conditions',3:'Underlying Conditions Pending',
                                                                                                       4:'Total Deaths'}, axis = 1).reset_index(drop=True)
            #temp.drop(['Unnamed: 1'], axis = 1, inplace = True)
        elif df.equals(nycc):
            temp = df[getIndices(df,categories[i])[0][0]:getIndices(df,categories[i+1])[0][0]].rename({'.':categories[i]}, axis = 1).reset_index(drop=True)
            temp.drop(['Unnamed: 1'], axis = 1, inplace = True)
        
        
        temp.drop(0, axis = 0, inplace = True)

        temp[categories[i]] = temp[categories[i]].map(lambda x: x[1:].lstrip())
        temp.iloc[:,1] = temp.iloc[:,1].apply(countform)
        temp['Date'] = today #today

        yield temp

**TODAY'S DATA**

In [59]:
#CREATE DATAFRAMES
age_death,sex_death,borough_death = getdfs(nycd,dcats)
age_hosp,sex_hosp,borough_hosp = getdfs(nych,hcats)
age_cases,cases50new,sex_cases,borough_cases = getdfs(nycc,ccats)

In [60]:
#combines  several of the new dataframes into one
def mergedfs(leftdf,rightdf,on):
    temp = pd.merge(leftdf,rightdf, how='left', on = on)
    temp.rename({'Date_x':'Date'},axis = 1, inplace = True)
    temp.drop(['Date_y'], axis=1, inplace = True)
    temp = temp.fillna(0)
    return temp
    

In [61]:
age_death

,Age Group,Underlying Conditions,No Underlying Conditions,Underlying Conditions Pending,Total Deaths,Date
1,0 to 17,1,0.0,0.0,1.0,2020-03-31
2,18 to 44,44,2.0,11.0,57.0,2020-03-31
3,45 to 64,180,9.0,29.0,218.0,2020-03-31
4,65 to 74,182,0.0,39.0,221.0,2020-03-31
5,75 and over,341,3.0,91.0,435.0,2020-03-31


In [62]:
age_hosp

,Age Group,Ever Hospitalized Cases,Total Cases,Date
1,0 to 17,72,714.0,2020-03-31
2,18 to 44,1448,16028.0,2020-03-31
3,45 to 64,2887,13344.0,2020-03-31
4,65 to 74,1612,4496.0,2020-03-31
5,75 and over,1722,3410.0,2020-03-31
6,Unknown,0,95.0,2020-03-31


In [63]:
age_cases

,Age Group,Total Cases,Date
1,0 to 17,748,2020-03-31
2,18 to 44,17038,2020-03-31
3,45 to 64,14395,2020-03-31
4,65 to 74,4883,2020-03-31
5,75 and over,3739,2020-03-31
6,Unknown,97,2020-03-31


In [64]:
age_combined = mergedfs(age_hosp,age_death,'Age Group')
age_combined

,Age Group,Ever Hospitalized Cases,Total Cases,Date,Underlying Conditions,No Underlying Conditions,Underlying Conditions Pending,Total Deaths
0,0 to 17,72,714.0,2020-03-31,1.0,0.0,0.0,1.0
1,18 to 44,1448,16028.0,2020-03-31,44.0,2.0,11.0,57.0
2,45 to 64,2887,13344.0,2020-03-31,180.0,9.0,29.0,218.0
3,65 to 74,1612,4496.0,2020-03-31,182.0,0.0,39.0,221.0
4,75 and over,1722,3410.0,2020-03-31,341.0,3.0,91.0,435.0
5,Unknown,0,95.0,2020-03-31,0.0,0.0,0.0,0.0


+++++++++++++++++++++++++++++++++
+++++++++++++++++++++++++++++++++

In [65]:
cases50new

,Age 50 and over,Total Cases,Date
1,Yes,19481,2020-03-31
2,No,21322,2020-03-31


In [66]:
sex_death

,Sex,Underlying Conditions,No Underlying Conditions,Underlying Conditions Pending,Total Deaths,Date
1,Female,285,4.0,63.0,352.0,2020-03-31
2,Male,462,10.0,107.0,579.0,2020-03-31
3,Unknown,1,0.0,0.0,1.0,2020-03-31


In [67]:
sex_hosp

,Sex,Ever Hospitalized Cases,Total Cases,Date
1,Female,3130,16920.0,2020-03-31
2,Male,4610,21120.0,2020-03-31
3,Unknown,1,47.0,2020-03-31


In [68]:
sex_cases

,Sex,Total Cases,Date
1,Female,18264,2020-03-31
2,Male,22587,2020-03-31
3,Unknown,49,2020-03-31


In [69]:
sex_combined = mergedfs(sex_hosp,sex_death,'Sex')
sex_combined

,Sex,Ever Hospitalized Cases,Total Cases,Date,Underlying Conditions,No Underlying Conditions,Underlying Conditions Pending,Total Deaths
0,Female,3130,16920.0,2020-03-31,285,4.0,63.0,352.0
1,Male,4610,21120.0,2020-03-31,462,10.0,107.0,579.0
2,Unknown,1,47.0,2020-03-31,1,0.0,0.0,1.0


+++++++++++++++++++++++++++++++++
+++++++++++++++++++++++++++++++++

In [70]:
borough_death

,Borough,Underlying Conditions,No Underlying Conditions,Underlying Conditions Pending,Total Deaths,Date
1,Bronx,210,1.0,19.0,230.0,2020-03-31
2,Brooklyn,163,4.0,48.0,215.0,2020-03-31
3,Manhattan,92,1.0,26.0,119.0,2020-03-31
4,Queens,238,8.0,63.0,309.0,2020-03-31
5,Staten Island,44,0.0,14.0,58.0,2020-03-31
6,Unknown,1,0.0,0.0,1.0,2020-03-31


In [71]:
borough_hosp

,Borough,Ever Hospitalized Cases,Total Cases,Date
1,Bronx,1880,6925.0,2020-03-31
2,Brooklyn,1661,10171.0,2020-03-31
3,Manhattan,1075,6060.0,2020-03-31
4,Queens,2650,12756.0,2020-03-31
5,Staten Island,465,2140.0,2020-03-31
6,Unknown,10,35.0,2020-03-31


In [72]:
borough_cases

,Borough,Total Cases,Date
1,Bronx,7625,2020-03-31
2,Brooklyn,10904,2020-03-31
3,Manhattan,6446,2020-03-31
4,Queens,13576,2020-03-31
5,Staten Island,2314,2020-03-31
6,Unknown,35,2020-03-31


In [73]:
borough_combined = mergedfs(borough_hosp,borough_death,'Borough')
borough_combined

,Borough,Ever Hospitalized Cases,Total Cases,Date,Underlying Conditions,No Underlying Conditions,Underlying Conditions Pending,Total Deaths
0,Bronx,1880,6925.0,2020-03-31,210,1.0,19.0,230.0
1,Brooklyn,1661,10171.0,2020-03-31,163,4.0,48.0,215.0
2,Manhattan,1075,6060.0,2020-03-31,92,1.0,26.0,119.0
3,Queens,2650,12756.0,2020-03-31,238,8.0,63.0,309.0
4,Staten Island,465,2140.0,2020-03-31,44,0.0,14.0,58.0
5,Unknown,10,35.0,2020-03-31,1,0.0,0.0,1.0


In [147]:
#underlying_death

#### READ IN PREVIOUS CSVs

**AGE**

In [52]:
age = pd.read_csv(folderpath / 'age.csv')
#age = age.rename({'Count':'Total Deaths'}, axis = 1)
age

,Age Group,Ever Hospitalized Cases,Total Cases,Date,Underlying Conditions,No Underlying Conditions,Underlying Conditions Pending,Total Deaths
0,0 to 17,67,619.0,2020-03-29,1.0,0.0,0.0,1.0
1,18 to 44,1459,14233.0,2020-03-29,28.0,1.0,10.0,39.0
2,45 to 64,2765,11577.0,2020-03-29,140.0,8.0,37.0,185.0
3,65 to 74,1499,3954.0,2020-03-29,152.0,1.0,30.0,183.0
4,75 and over,1620,3020.0,2020-03-29,303.0,3.0,62.0,368.0
5,Unknown,0,71.0,2020-03-29,0.0,0.0,0.0,0.0
6,0 to 17,72,714.0,2020-03-30 00:00:00,1.0,0.0,0.0,1.0
7,18 to 44,1448,16028.0,2020-03-30 00:00:00,39.0,2.0,13.0,54.0
8,45 to 64,2887,13344.0,2020-03-30 00:00:00,170.0,9.0,37.0,216.0
9,65 to 74,1612,4496.0,2020-03-30 00:00:00,167.0,0.0,48.0,215.0


In [177]:
# newcols =['Underlying Conditions','No Underlying Conditions','Underlying Conditions Pending']

# for cols in newcols:
#     age[cols] = 0

In [179]:
# rearrange = ['Age Group','Underlying Conditions','No Underlying Conditions','Underlying Conditions Pending',
#              'Total Deaths','Date']

# age = age[rearrange]

In [28]:
# newcols =['Ever Hospitalized Cases','Total Cases','Total Deaths']

# for cols in newcols:
#     age[cols] = 0

In [58]:
age_combined

,Age Group,Ever Hospitalized Cases,Total Cases,Date,Underlying Conditions,No Underlying Conditions,Underlying Conditions Pending,Total Deaths
0,0 to 17,72,714.0,2020-03-30,1.0,0.0,0.0,1.0
1,18 to 44,1448,16028.0,2020-03-30,39.0,2.0,13.0,54.0
2,45 to 64,2887,13344.0,2020-03-30,170.0,9.0,37.0,216.0
3,65 to 74,1612,4496.0,2020-03-30,167.0,0.0,48.0,215.0
4,75 and over,1722,3410.0,2020-03-30,327.0,3.0,98.0,428.0
5,Unknown,0,95.0,2020-03-30,0.0,0.0,0.0,0.0


In [80]:
agenew = pd.concat([age,age_combined], axis = 0, sort = False).reset_index(drop = True)
agenew

,Age Group,Ever Hospitalized Cases,Total Cases,Date,Underlying Conditions,No Underlying Conditions,Underlying Conditions Pending,Total Deaths
0,0 to 17,67,619.0,2020-03-29,1.0,0.0,0.0,1.0
1,18 to 44,1459,14233.0,2020-03-29,28.0,1.0,10.0,39.0
2,45 to 64,2765,11577.0,2020-03-29,140.0,8.0,37.0,185.0
3,65 to 74,1499,3954.0,2020-03-29,152.0,1.0,30.0,183.0
4,75 and over,1620,3020.0,2020-03-29,303.0,3.0,62.0,368.0
5,Unknown,0,71.0,2020-03-29,0.0,0.0,0.0,0.0
6,0 to 17,72,714.0,2020-03-30 00:00:00,1.0,0.0,0.0,1.0
7,18 to 44,1448,16028.0,2020-03-30 00:00:00,39.0,2.0,13.0,54.0
8,45 to 64,2887,13344.0,2020-03-30 00:00:00,170.0,9.0,37.0,216.0
9,65 to 74,1612,4496.0,2020-03-30 00:00:00,167.0,0.0,48.0,215.0


**BOROUGH**

In [60]:
borough = pd.read_csv('borough.csv')
borough = borough.rename({'Count':'Deaths'}, axis = 1)
borough

,Borough,Ever Hospitalized Cases,Total Cases,Date,Underlying Conditions,No Underlying Conditions,Underlying Conditions Pending,Total Deaths
0,Bronx,1729,6250.0,2020-03-29,176,0.0,12.0,188.0
1,Brooklyn,1617,8887.0,2020-03-29,136,5.0,44.0,185.0
2,Manhattan,1074,5582.0,2020-03-29,82,1.0,20.0,103.0
3,Queens,2536,10737.0,2020-03-29,194,7.0,52.0,253.0
4,Staten Island,444,1984.0,2020-03-29,35,0.0,11.0,46.0
5,Unknown,10,34.0,2020-03-29,1,0.0,0.0,1.0


In [74]:
def assemble_df_new(oldcsv,newdf): #newcols
    #for cols in newcols:
     #   olddf[cols] = 0
    olddf = pd.read_csv(folderpath / oldcsv)
    temp = pd.concat([olddf,newdf], axis = 0, sort = False).reset_index(drop = True)
    #temp = temp.fillna(0)
    return temp

In [75]:
borough_combined

,Borough,Ever Hospitalized Cases,Total Cases,Date,Underlying Conditions,No Underlying Conditions,Underlying Conditions Pending,Total Deaths
0,Bronx,1880,6925.0,2020-03-31,210,1.0,19.0,230.0
1,Brooklyn,1661,10171.0,2020-03-31,163,4.0,48.0,215.0
2,Manhattan,1075,6060.0,2020-03-31,92,1.0,26.0,119.0
3,Queens,2650,12756.0,2020-03-31,238,8.0,63.0,309.0
4,Staten Island,465,2140.0,2020-03-31,44,0.0,14.0,58.0
5,Unknown,10,35.0,2020-03-31,1,0.0,0.0,1.0


In [76]:
boroughnew = assemble_df_new('borough.csv',borough_combined)
boroughnew

,Borough,Ever Hospitalized Cases,Total Cases,Date,Underlying Conditions,No Underlying Conditions,Underlying Conditions Pending,Total Deaths
0,Bronx,1729,6250.0,2020-03-29,176,0.0,12.0,188.0
1,Brooklyn,1617,8887.0,2020-03-29,136,5.0,44.0,185.0
2,Manhattan,1074,5582.0,2020-03-29,82,1.0,20.0,103.0
3,Queens,2536,10737.0,2020-03-29,194,7.0,52.0,253.0
4,Staten Island,444,1984.0,2020-03-29,35,0.0,11.0,46.0
5,Unknown,10,34.0,2020-03-29,1,0.0,0.0,1.0
6,Bronx,1880,6925.0,2020-03-30 00:00:00,198,1.0,16.0,215.0
7,Brooklyn,1661,10171.0,2020-03-30 00:00:00,153,4.0,59.0,216.0
8,Manhattan,1075,6060.0,2020-03-30 00:00:00,90,1.0,28.0,119.0
9,Queens,2650,12756.0,2020-03-30 00:00:00,222,8.0,75.0,305.0


**SEX**

In [67]:
# sex = pd.read_csv('sex.csv')
# sex = sex.rename({'Count':'Deaths'}, axis = 1)
# sex

,Sex,Ever Hospitalized Cases,Total Cases,Date,Underlying Conditions,No Underlying Conditions,Underlying Conditions Pending,Total Deaths
0,Female,3001,14837.0,2020-03-29,246.0,5.0,51.0,302.0
1,Male,4408,18593.0,2020-03-29,378.0,8.0,88.0,474.0
2,Unknown,1,44.0,2020-03-29,0.0,0.0,0.0,0.0


In [68]:
sex_combined

,Sex,Ever Hospitalized Cases,Total Cases,Date,Underlying Conditions,No Underlying Conditions,Underlying Conditions Pending,Total Deaths
0,Female,3130,16920.0,2020-03-30,270,4.0,70.0,344.0
1,Male,4610,21120.0,2020-03-30,433,10.0,126.0,569.0
2,Unknown,1,47.0,2020-03-30,1,0.0,0.0,1.0


In [69]:
sexnew = assemble_df_new('sex.csv',sex_combined)
sexnew

,Sex,Ever Hospitalized Cases,Total Cases,Date,Underlying Conditions,No Underlying Conditions,Underlying Conditions Pending,Total Deaths
0,Female,3001,14837.0,2020-03-29,246.0,5.0,51.0,302.0
1,Male,4408,18593.0,2020-03-29,378.0,8.0,88.0,474.0
2,Unknown,1,44.0,2020-03-29,0.0,0.0,0.0,0.0
3,Female,3130,16920.0,2020-03-30 00:00:00,270.0,4.0,70.0,344.0
4,Male,4610,21120.0,2020-03-30 00:00:00,433.0,10.0,126.0,569.0
5,Unknown,1,47.0,2020-03-30 00:00:00,1.0,0.0,0.0,1.0


**CASES50**

In [70]:
cases50current = pd.read_csv('cases50.csv')
cases50current

,Age 50 and over,Total Cases,Date
0,Yes,15706,2020-03-29
1,No,17697,2020-03-29


In [71]:
cases50new

,Age 50 and over,Total Cases,Date
1,Yes,17958,2020-03-30
2,No,20034,2020-03-30


In [78]:
cases50current = assemble_df_new('cases50.csv',cases50new)
cases50current

,Age 50 and over,Total Cases,Date
0,Yes,15706,2020-03-29
1,No,17697,2020-03-29
2,Yes,17958,2020-03-30 00:00:00
3,No,20034,2020-03-30 00:00:00
4,Yes,19481,2020-03-31
5,No,21322,2020-03-31


In [110]:
# underlying = pd.read_csv('underlying.csv')
# underlying = underlying.rename({'Count':'Deaths'}, axis = 1)
# underlying

**OUTPUT CSVS**

In [28]:
#only run when starting fresh
# age_combined.to_csv('age.csv', index = False)
# sex_combined.to_csv('sex.csv', index = False)
# borough_combined.to_csv('borough.csv', index = False)
# cases50new.to_csv('cases50.csv', index = False)

In [81]:
agenew.to_csv(folderpath / 'testage.csv', index = False)
sexnew.to_csv(folderpath / 'sex.csv', index = False)
boroughnew.to_csv(folderpath / 'borough.csv', index = False)
cases50current.to_csv(folderpath / 'cases50.csv', index = False)

In [82]:
agenew

,Age Group,Ever Hospitalized Cases,Total Cases,Date,Underlying Conditions,No Underlying Conditions,Underlying Conditions Pending,Total Deaths
0,0 to 17,67,619.0,2020-03-29,1.0,0.0,0.0,1.0
1,18 to 44,1459,14233.0,2020-03-29,28.0,1.0,10.0,39.0
2,45 to 64,2765,11577.0,2020-03-29,140.0,8.0,37.0,185.0
3,65 to 74,1499,3954.0,2020-03-29,152.0,1.0,30.0,183.0
4,75 and over,1620,3020.0,2020-03-29,303.0,3.0,62.0,368.0
5,Unknown,0,71.0,2020-03-29,0.0,0.0,0.0,0.0
6,0 to 17,72,714.0,2020-03-30 00:00:00,1.0,0.0,0.0,1.0
7,18 to 44,1448,16028.0,2020-03-30 00:00:00,39.0,2.0,13.0,54.0
8,45 to 64,2887,13344.0,2020-03-30 00:00:00,170.0,9.0,37.0,216.0
9,65 to 74,1612,4496.0,2020-03-30 00:00:00,167.0,0.0,48.0,215.0


In [89]:
sex = pd.read_csv('/Users/Emanuel/Documents/2019 Plan/NYC Data Science Academy/Post Bootcamp/COVID-19/Forecasting/covid19-global-forecasting-week-1/nyc/sex.csv')
sex

,Sex,Ever Hospitalized Cases,Total Cases,Date,Underlying Conditions,No Underlying Conditions,Underlying Conditions Pending,Total Deaths
0,Female,3001,14837.0,2020-03-29,246.0,5.0,51.0,302.0
1,Male,4408,18593.0,2020-03-29,378.0,8.0,88.0,474.0
2,Unknown,1,44.0,2020-03-29,0.0,0.0,0.0,0.0
3,Female,3130,16920.0,2020-03-30 00:00:00,270.0,4.0,70.0,344.0
4,Male,4610,21120.0,2020-03-30 00:00:00,433.0,10.0,126.0,569.0
5,Unknown,1,47.0,2020-03-30 00:00:00,1.0,0.0,0.0,1.0
6,Female,3130,16920.0,2020-03-31,285.0,4.0,63.0,352.0
7,Male,4610,21120.0,2020-03-31,462.0,10.0,107.0,579.0
8,Unknown,1,47.0,2020-03-31,1.0,0.0,0.0,1.0
9,Female,3130,16920.0,2020-03-31,285.0,4.0,63.0,352.0


In [ ]:
csvs = ['/Users/Emanuel/Documents/2019 Plan/NYC Data Science Academy/Post Bootcamp/COVID-19/Forecasting/covid19-global-forecasting-week-1/nyc/age.csv',
        '/Users/Emanuel/Documents/2019 Plan/NYC Data Science Academy/Post Bootcamp/COVID-19/Forecasting/covid19-global-forecasting-week-1/nyc/sex.csv',
        '/Users/Emanuel/Documents/2019 Plan/NYC Data Science Academy/Post Bootcamp/COVID-19/Forecasting/covid19-global-forecasting-week-1/nyc/borough.csv',
        '/Users/Emanuel/Documents/2019 Plan/NYC Data Science Academy/Post Bootcamp/COVID-19/Forecasting/covid19-global-forecasting-week-1/nyc/cases50.csv']

### II. US National Level Data

### III. Global Data

In [3]:
! rm time_series_covid19_deaths_global.csv
! rm time_series_covid19_confirmed_global.csv
! rm time_series_covid19_recovered_global.csv

urls = ['https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv',
        'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv',
        'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv']

for url in urls:
    filename = wget.download(url)

In [4]:
confirmed = pd.read_csv('time_series_covid19_confirmed_global.csv')
deaths = pd.read_csv('time_series_covid19_deaths_global.csv')
recovered = pd.read_csv('time_series_covid19_recovered_global.csv') 

In [5]:
confirmed.sample(5)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20
244,NaN,Saint Kitts and Nevis,17.357822,-62.782998,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,2,2,2,7
32,NaN,Cabo Verde,16.538800,-23.041800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,3,3,3,3,4,4,5,5,6,6
9,New South Wales,Australia,-33.868800,151.209300,0,0,0,0,3,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,6,6,13,22,22,26,28,38,48,55,65,65,92,112,134,171,210,267,307,353,436,669,669,818,1029,1219,1405,1617,1791,2032
110,Mayotte,France,-12.827500,45.166200,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,3,3,6,7,11,24,36,36,36,50,63,63,82
146,NaN,Latvia,56.879600,24.603200,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,2,6,8,10,10,17,26,30,34,49,71,86,111,124,139,180,197,221,244,280,305,347,376


In [ ]:
#confirmed[(confirmed['Province/State'].str.contains('York') == True)|(confirmed['Province/State'].str.contains(', NY') == True)]

In [ ]:
pd.melt?

In [ ]:
np.any?

In [6]:
dates = confirmed.columns[4:]
dates[-5:]
print('The latest data is as of {}.'.format(dates[-1]))

The latest data is as of 3/30/20.


**Convert the dataframes from multiple dates across columns to 1 column containing all the dates. This is for ease of time-series modeling later.**

In [7]:
dates

Index(['1/22/20', '1/23/20', '1/24/20', '1/25/20', '1/26/20', '1/27/20',
       '1/28/20', '1/29/20', '1/30/20', '1/31/20', '2/1/20', '2/2/20',
       '2/3/20', '2/4/20', '2/5/20', '2/6/20', '2/7/20', '2/8/20', '2/9/20',
       '2/10/20', '2/11/20', '2/12/20', '2/13/20', '2/14/20', '2/15/20',
       '2/16/20', '2/17/20', '2/18/20', '2/19/20', '2/20/20', '2/21/20',
       '2/22/20', '2/23/20', '2/24/20', '2/25/20', '2/26/20', '2/27/20',
       '2/28/20', '2/29/20', '3/1/20', '3/2/20', '3/3/20', '3/4/20', '3/5/20',
       '3/6/20', '3/7/20', '3/8/20', '3/9/20', '3/10/20', '3/11/20', '3/12/20',
       '3/13/20', '3/14/20', '3/15/20', '3/16/20', '3/17/20', '3/18/20',
       '3/19/20', '3/20/20', '3/21/20', '3/22/20', '3/23/20', '3/24/20',
       '3/25/20', '3/26/20', '3/27/20', '3/28/20', '3/29/20', '3/30/20'],
      dtype='object')

In [8]:
confirmed = confirmed.melt(id_vars=['Province/State','Country/Region','Lat','Long'], value_vars=dates, var_name = 'Date',value_name = 'Confirmed')
deaths = deaths.melt(id_vars=['Province/State','Country/Region','Lat','Long'], value_vars=dates, var_name = 'Date',value_name = 'Deaths')
recovered = recovered.melt(id_vars=['Province/State','Country/Region','Lat','Long'], value_vars=dates, var_name = 'Date',value_name = 'Recovered')

In [9]:
#confirmed.head()
#deaths.head()
recovered.head()

,Province/State,Country/Region,Lat,Long,Date,Recovered
0,NaN,Afghanistan,33.0000,65.0000,1/22/20,0
1,NaN,Albania,41.1533,20.1683,1/22/20,0
2,NaN,Algeria,28.0339,1.6596,1/22/20,0
3,NaN,Andorra,42.5063,1.5218,1/22/20,0
4,NaN,Angola,-11.2027,17.8739,1/22/20,0


**Make a combined dataframe**

In [10]:
#combine confirmed,deaths, and recovered into 1 dataframe.

combined = pd.concat([confirmed,deaths['Deaths'],recovered['Recovered']], axis = 1, sort = False)
combined.sample(10)

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
14086,NaN,France,46.227600,2.213700,3/17/20,7652,148,2.0
4996,NaN,New Zealand,-40.900600,174.886000,2/10/20,0,0,0.0
3238,NaN,Saint Vincent and the Grenadines,12.984300,-61.287200,2/3/20,0,0,0.0
1154,NaN,Jamaica,18.109600,-77.297500,1/26/20,0,0,0.0
6851,NaN,Kosovo,42.602636,20.902977,2/17/20,0,0,0.0
5823,NaN,Syria,34.802075,38.996815,2/13/20,0,0,60.0
5185,NaN,Fiji,-17.713400,178.065000,2/11/20,0,0,0.0
9473,Shanxi,China,37.577700,112.292200,2/28/20,133,0,0.0
16149,NaN,Lebanon,33.854700,35.862300,3/25/20,333,6,1.0
979,Bermuda,United Kingdom,32.307800,-64.750500,1/25/20,0,0,0.0


In [11]:
#format 'Date' column to date time and then reformat to mm/dd/yy format.
combined['Date'] = pd.to_datetime(combined['Date'], dayfirst = False, yearfirst = False)
combined['Date']= combined['Date'].dt.strftime('%m/%d/%Y')
latest = max(combined['Date'])

In [12]:
combined['Country/Region'].unique()

array(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia',
       'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh',
       'Barbados', 'Belarus', 'Belgium', 'Benin', 'Bhutan', 'Bolivia',
       'Bosnia and Herzegovina', 'Brazil', 'Brunei', 'Bulgaria',
       'Burkina Faso', 'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada',
       'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia',
       'Congo (Brazzaville)', 'Congo (Kinshasa)', 'Costa Rica',
       "Cote d'Ivoire", 'Croatia', 'Diamond Princess', 'Cuba', 'Cyprus',
       'Czechia', 'Denmark', 'Djibouti', 'Dominican Republic', 'Ecuador',
       'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia',
       'Eswatini', 'Ethiopia', 'Fiji', 'Finland', 'France', 'Gabon',
       'Gambia', 'Georgia', 'Germany', 'Ghana', 'Greece', 'Guatemala',
       'Guinea', 'Guyana', 'Haiti', 'Holy See', 'Honduras', 'Hungary',
       'Iceland', 'India

In [13]:
#change Korea, South to South Korea as North Korea does not report its cases.

combined.loc[:,'Country/Region'] = combined.loc[:,'Country/Region'].replace('Korea, South','South Korea')

In [ ]:
# #make a US-only dataframe. Can use it for analysis later.

# us = combined.loc[combined['Country/Region'] == 'US']

In [14]:
#back to the global combined. Take out the state level detail because this is global data.
#combined = combined.loc[combined['Province/State'].str.contains(',') != True]
combined.sample(30)

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
12206,Victoria,Australia,-37.8136,144.9631,03/10/2020,18,0,20.0
1442,NaN,Niger,17.6078,8.0817,01/27/2020,0,0,0.0
14485,NaN,Armenia,40.0691,45.0382,03/19/2020,115,0,1.0
10434,NaN,Bangladesh,23.6850,90.3563,03/03/2020,0,0,0.0
2105,Shandong,China,36.3427,118.1498,01/30/2020,158,0,0.0
9289,NaN,Kyrgyzstan,41.2044,74.7661,02/27/2020,0,0,0.0
14011,Nova Scotia,Canada,44.6820,-63.7443,03/17/2020,7,0,0.0
16307,Chongqing,China,30.0572,107.8740,03/26/2020,578,6,39.0
3596,Newfoundland and Labrador,Canada,53.1355,-57.6604,02/05/2020,0,0,0.0
5874,NaN,Cabo Verde,16.5388,-23.0418,02/14/2020,0,0,0.0


In [15]:
#Fatality Rate and Recovered
combined.loc[:,'Fatality Rate'] = combined['Deaths']/combined['Confirmed']
combined.loc[:,'Fatality Rate'] = combined.loc[:,'Fatality Rate'].fillna(0)
combined.loc[:,'Recovered'] = combined.loc[:,'Recovered'].fillna(0)

In [16]:
#Fill NAs for Province/State so the groupby operation later will work.
combined.loc[:,'Province/State'] = combined.loc[:,'Province/State'].fillna('Unavailable')

In [17]:

combined[(combined['Confirmed']>100) & (combined['Date'] == latest)].sort_values(['Date','Fatality Rate'], ascending = False)[:10]

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Fatality Rate
17409,Unavailable,Italy,43.0000,12.0000,03/30/2020,101739,11591,0.0,0.113929
17463,Unavailable,San Marino,43.9424,12.4578,03/30/2020,230,25,0.0,0.108696
17473,Unavailable,Spain,40.0000,-4.0000,03/30/2020,87956,7716,0.0,0.087726
17404,Unavailable,Indonesia,-0.7893,113.9213,03/30/2020,1414,122,0.0,0.086280
17441,Unavailable,Netherlands,52.1326,5.2913,03/30/2020,11750,864,0.0,0.073532
17406,Unavailable,Iraq,33.0000,44.0000,03/30/2020,630,46,0.0,0.073016
17388,Unavailable,France,46.2276,2.2137,03/30/2020,44550,3024,0.0,0.067879
17405,Unavailable,Iran,32.0000,53.0000,03/30/2020,41495,2757,0.0,0.066442
17495,Unavailable,United Kingdom,55.3781,-3.4360,03/30/2020,22141,1408,0.0,0.063592
17370,Unavailable,Egypt,26.0000,30.0000,03/30/2020,656,41,0.0,0.062500


In [99]:
combined[(combined['Country/Region']=='India') & (combined['Date']==latest)]

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Fatality Rate
15566,Unavailable,India,21.0,78.0,03/25/2020,657,12,0.018265


In [18]:
combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17526 entries, 0 to 17525
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Province/State  17526 non-null  object 
 1   Country/Region  17526 non-null  object 
 2   Lat             17526 non-null  float64
 3   Long            17526 non-null  float64
 4   Date            17526 non-null  object 
 5   Confirmed       17526 non-null  int64  
 6   Deaths          17526 non-null  int64  
 7   Recovered       17526 non-null  float64
 8   Fatality Rate   17526 non-null  float64
dtypes: float64(4), int64(2), object(3)
memory usage: 1.2+ MB


In [ ]:

combined[(combined['Date']=='2/12/20') & (combined['Province/State']=='Hubei')]

In [19]:
#insert a RowID index to track position of items.
combined.insert(0,'RowID',range(1,len(combined.index)+1))

In [20]:
combined.head()

,RowID,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Fatality Rate
0,1,Unavailable,Afghanistan,33.0000,65.0000,01/22/2020,0,0,0.0,0.0
1,2,Unavailable,Albania,41.1533,20.1683,01/22/2020,0,0,0.0,0.0
2,3,Unavailable,Algeria,28.0339,1.6596,01/22/2020,0,0,0.0,0.0
3,4,Unavailable,Andorra,42.5063,1.5218,01/22/2020,0,0,0.0,0.0
4,5,Unavailable,Angola,-11.2027,17.8739,01/22/2020,0,0,0.0,0.0


In [21]:
#combined.drop('RowID',axis=1,inplace=True)
combined.shape

(17526, 10)

In [22]:
combined.tail()

,RowID,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Fatality Rate
17521,17522,Anguilla,United Kingdom,18.2206,-63.0686,03/30/2020,2,0,0.0,0.0
17522,17523,British Virgin Islands,United Kingdom,18.4207,-64.6400,03/30/2020,2,0,0.0,0.0
17523,17524,Turks and Caicos Islands,United Kingdom,21.6940,-71.7979,03/30/2020,5,0,0.0,0.0
17524,17525,Unavailable,MS Zaandam,0.0000,0.0000,03/30/2020,2,0,0.0,0.0
17525,17526,Unavailable,Botswana,-22.3285,24.6849,03/30/2020,3,0,0.0,0.0


### Create a dataframe grouped by Countries

In [23]:
#does NOT have state/provice granularity.
countries = combined.groupby(['Date','Country/Region'])[['RowID','Date','Country/Region','Confirmed','Deaths','Recovered']].sum().reset_index()
countries

,Date,Country/Region,RowID,Confirmed,Deaths,Recovered
0,01/22/2020,Afghanistan,1,0,0,0.0
1,01/22/2020,Albania,2,0,0,0.0
2,01/22/2020,Algeria,3,0,0,0.0
3,01/22/2020,Andorra,4,0,0,0.0
4,01/22/2020,Angola,5,0,0,0.0
...,...,...,...,...,...,...
12277,03/30/2020,Venezuela,17500,135,3,0.0
12278,03/30/2020,Vietnam,17501,203,0,0.0
12279,03/30/2020,West Bank and Gaza,17514,116,1,0.0
12280,03/30/2020,Zambia,17502,35,0,0.0


In [24]:
countries.shape

(12282, 6)

In [25]:

print(countries['Date'].unique())
print('\n')
print('latest date is {}'.format(latest))

['01/22/2020' '01/23/2020' '01/24/2020' '01/25/2020' '01/26/2020'
 '01/27/2020' '01/28/2020' '01/29/2020' '01/30/2020' '01/31/2020'
 '02/01/2020' '02/02/2020' '02/03/2020' '02/04/2020' '02/05/2020'
 '02/06/2020' '02/07/2020' '02/08/2020' '02/09/2020' '02/10/2020'
 '02/11/2020' '02/12/2020' '02/13/2020' '02/14/2020' '02/15/2020'
 '02/16/2020' '02/17/2020' '02/18/2020' '02/19/2020' '02/20/2020'
 '02/21/2020' '02/22/2020' '02/23/2020' '02/24/2020' '02/25/2020'
 '02/26/2020' '02/27/2020' '02/28/2020' '02/29/2020' '03/01/2020'
 '03/02/2020' '03/03/2020' '03/04/2020' '03/05/2020' '03/06/2020'
 '03/07/2020' '03/08/2020' '03/09/2020' '03/10/2020' '03/11/2020'
 '03/12/2020' '03/13/2020' '03/14/2020' '03/15/2020' '03/16/2020'
 '03/17/2020' '03/18/2020' '03/19/2020' '03/20/2020' '03/21/2020'
 '03/22/2020' '03/23/2020' '03/24/2020' '03/25/2020' '03/26/2020'
 '03/27/2020' '03/28/2020' '03/29/2020' '03/30/2020']


latest date is 03/30/2020


In [80]:
#countries['Country/Region'].unique()

In [26]:
(countries['Confirmed'].sum() == combined['Confirmed'].sum()) and (countries['Deaths'].sum() == combined['Deaths'].sum())

True

In [27]:
combined[combined['Date'] == latest]['Confirmed'].sum()

782365

In [28]:
countries[countries['Date'] == latest].sort_values(['Confirmed'], ascending = False)

,Date,Country/Region,RowID,Confirmed,Deaths,Recovered
12270,03/30/2020,US,17498,161807,2978,0.0
12187,03/30/2020,Italy,17410,101739,11591,0.0
12255,03/30/2020,Spain,17474,87956,7716,0.0
12139,03/30/2020,China,572154,82198,3308,0.0
12168,03/30/2020,Germany,17393,66885,645,0.0
12164,03/30/2020,France,173845,45170,3030,0.0
12183,03/30/2020,Iran,17406,41495,2757,0.0
12274,03/30/2020,United Kingdom,175020,22453,1411,0.0
12260,03/30/2020,Switzerland,17479,15922,359,0.0
12120,03/30/2020,Belgium,17296,11899,513,0.0


In [29]:
world_graph = countries.groupby(['Date'])[['Confirmed','Deaths','Recovered']].sum().reset_index()
#world_graph = world_graph.melt(id_vars=['Date'], value_vars=['Confirmed','Deaths'], var_name = 'Case',value_name = 'Count')

In [30]:
world_graph['Date']=pd.to_datetime(world_graph['Date'])

In [31]:
world_graph.tail(16)

,Date,Confirmed,Deaths,Recovered
53,2020-03-15,167454,6440,83388.0
54,2020-03-16,181574,7126,89486.0
55,2020-03-17,197102,7905,151723.0
56,2020-03-18,214821,8733,95667.0
57,2020-03-19,242500,9867,98750.0
58,2020-03-20,272035,11299,99317.0
59,2020-03-21,304396,12973,116533.0
60,2020-03-22,336953,14651,139254.0
61,2020-03-23,378235,16505,131030.0
62,2020-03-24,418045,18625,136452.0


In [219]:
#world_graph1 = world_graph.pivot(index='Date', columns = 'Case')['Count'].reset_index()

In [32]:
world_graph.loc[:,'New Deaths'] = world_graph.loc[:,'Deaths']
world_graph.loc[0,'New Deaths'] = 0
for i in range(1,len(world_graph['Deaths'])):
    world_graph.loc[i,'New Deaths'] = world_graph.loc[i,'Deaths'] - world_graph.loc[i-1,'Deaths']
    
world_graph.rename({'Deaths':'Total Deaths'}, axis =1, inplace=True)

In [33]:
world_graph['Fatality Rate'] = round((world_graph['Total Deaths']/world_graph['Confirmed'])*100,2)

In [34]:
world_graph.head(10)

,Date,Confirmed,Total Deaths,Recovered,New Deaths,Fatality Rate
0,2020-01-22,555,17,28.0,0,3.06
1,2020-01-23,654,18,30.0,1,2.75
2,2020-01-24,941,26,38.0,8,2.76
3,2020-01-25,1434,42,83.0,16,2.93
4,2020-01-26,2118,56,63.0,14,2.64
5,2020-01-27,2927,82,105.0,26,2.80
6,2020-01-28,5578,131,126.0,49,2.35
7,2020-01-29,6166,133,143.0,2,2.16
8,2020-01-30,8234,171,222.0,38,2.08
9,2020-01-31,9927,213,284.0,42,2.15


In [35]:
np.any(world_graph['Fatality Rate'].isnull())

False

**RUN SELECTIVELY**

In [40]:
#create dataframes to prepare for graphing. One with Confirmed vs Total Deaths, and one for Total Deaths vs New Deaths.
world_graph1 = world_graph.melt(id_vars=['Date'], value_vars = ['Confirmed','Total Deaths','New Deaths'], var_name = 'Case', value_name = 'Count')

world_graphCvD = world_graph1[(world_graph1['Case'] == 'Confirmed')|(world_graph1['Case'] == 'Total Deaths')]
# world_graphD = world_graph1[(world_graph1['Case'] == 'New Deaths')|(world_graph1['Case'] == 'Total Deaths')]

===================

In [39]:
#plotly graph to show aggregate confirms vs deaths

# fig = px.area(world_graph1, x = "Date", y = "Count", color= 'Case',
#              title='Cases over time', color_discrete_sequence = ['red', 'green'])
# fig.show()

In [41]:
fig = px.line(world_graphCvD, x="Date", y="Count", color='Case',
             title='Cases over time: Line Plot', color_discrete_sequence = ['cyan', 'red'],template='plotly_dark')



fig.show()

In [42]:
fig = go.Figure()

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])


fig.add_trace(go.Scatter(x=world_graph['Date'], y=world_graph['Total Deaths'],
                    mode='lines',
                    name='Total Deaths',
                    line = dict(color='white', width=4),
                    hovertemplate =
                    '<i>Count</i>: %{y}'+
                    '<br><b>Date</b>: %{x}<br>',                    
                    showlegend = True),
                    secondary_y = False)
fig.add_trace(go.Scatter(x=world_graph['Date'], y=world_graph['New Deaths'],
                    mode='lines',
                    name='New Deaths',
                    line = dict(color='red'),
                    hovertemplate =
                    '<i>Count</i>: %{y}'+
                    '<br><b>Date</b>: %{x}<br>',                    
                    showlegend = True),
                    secondary_y = False)
fig.add_trace(go.Scatter(x=world_graph['Date'], y=world_graph['Fatality Rate'],
                    name='Fatality Rate',
                    line = dict(color='firebrick',dash='dash')),
                    secondary_y = True)

fig.update_layout(template='plotly_dark', title = 'Fatalities Since Jan 22 as of {}'.format(latest))

# Set x-axis title
fig.update_xaxes(title_text="Date")

# Set y-axes titles
fig.update_yaxes(title_text="Count", secondary_y=False)
fig.update_yaxes(title_text="% Rate", secondary_y=True)


fig.show()

In [43]:
# fig = px.line(world_graphD, x="Date", y="Count", color='Case',
#              title='Fatalities Since Jan 22', color_discrete_sequence = ['white', 'red'],template='plotly_dark')



# fig.show()